In [ ]:
import pandas as pd

# Load the potential matches

In [ ]:
df_genome = pd.read_csv('validation/final_unique_iphop_host_genome_pred_filtered.csv',sep=",")
df_genome.head()

# Map their name to our ids

In [ ]:
df_mapping = pd.read_csv("validation/ReadName2virOTU_ID.txt",sep=" ")
df_mapping.set_index('Read_name',inplace=True)
map_dict = df_mapping["virOTU"].to_dict()

In [ ]:
df_genome["virOTU"] = df_genome["Virus"].map(map_dict)
df_genome.head()

In [ ]:
print("Number of Host virus pairs:",df_genome.shape[0])

# CCMN

In [ ]:
df_ccm = pd.read_csv('tables/FramPhages2024_Hellinger_True_16_PV_CCM_CON_MAP_Network.csv',sep=";")
df_ccm.head()

# Add Taxa

In [ ]:
df_taxa = pd.read_csv('data/taxa_info.csv',sep=";")
df_taxa

In [ ]:
#df_taxa.fillna("virus",inplace=True)
df_taxa.loc[df_taxa['Kingdom'] == 'virus'] = df_taxa.loc[df_taxa['Kingdom'] == 'virus'].fillna('virus')
asv_fam =df_taxa[["ASV","Family"]]
asv_fam.set_index("ASV",inplace=True)
asv_fam_dict = asv_fam.to_dict()["Family"]
asv_fam_dict

In [ ]:
asv_kin =df_taxa[["ASV","Kingdom"]]
asv_kin.set_index("ASV",inplace=True)
asv_kin_dict = asv_kin.to_dict()["Kingdom"]
asv_kin_dict

In [ ]:
df_ccm["from_kingdom"] = df_ccm["from"].map(asv_kin_dict)
df_ccm["to_kingdom"]= df_ccm["to"].map(asv_kin_dict)

In [ ]:
df_ccm["from_family"] = df_ccm["from"].map(asv_fam_dict)
df_ccm["to_family"]= df_ccm["to"].map(asv_fam_dict)
df_ccm["InCCMN"] =1

In [ ]:
ccm_fams = df_ccm["from_family"].to_list() +df_ccm["to_family"].to_list()
ccm_virus = df_ccm["to"].to_list() +df_ccm["from"].to_list()
tab_fams =df_genome["Families"].to_list()
tab_vir =df_genome["virOTU"].to_list()

In [ ]:
df_genome[df_genome["virOTU"].isna()].to_csv("tables/not_found_virus_in_our_id.csv", sep=";", index=False)

# Overlapping Families

In [ ]:
print(len(set(ccm_fams)))
print(len(set(tab_fams)))
print(len(set(tab_fams).intersection(set(ccm_fams))))
print(len(set(ccm_fams).intersection(set(tab_fams))))
shared_fams = list(set(ccm_fams).intersection(set(tab_fams)))

# Overlapping Virus

In [ ]:
print(len(set(ccm_virus)))
print(len(set(tab_vir)))
print(len(set(tab_vir).intersection(set(ccm_virus))))
print(len(set(ccm_virus).intersection(set(tab_vir))))
shared_vir = list(set(ccm_virus).intersection(set(tab_vir)))

# Find in abundance table

In [ ]:
df_abundance = pd.read_csv("data/abundance.csv",sep=";")
df_abundance.set_index("Unnamed: 0",inplace=True)
all_asvs = df_abundance.columns.to_list()

In [ ]:
print(len(set(all_asvs).intersection(set(tab_vir))))
print("1 nan")
print("Sonst viren gleich => also durch CON raus")

In [ ]:
def return_fam(x):
    try:
        return asv_fam_dict[x]
    except:
        return "No"
all_famms = [return_fam(x) for x in all_asvs]
print(len(set(all_famms).intersection(set(tab_fams))))
print("Only 34 Familie of Bacteria, keine durchs con weg")

In [ ]:
df_genome_search = df_genome[df_genome[("Families")].isin(shared_fams) & df_genome["virOTU"].isin(shared_vir)]
df_genome_search

In [ ]:
df_ccm.head()

In [ ]:
asv_fam2 =df_taxa[["ASV","Family"]]
import numpy as np
bac_asv_list = list(set(asv_fam2[asv_fam2["Family"].isin(df_genome_search["Families"].to_list())]["ASV"].to_list()))
print("number of bac asvs",len(bac_asv_list))

In [ ]:
vir_asv_list = list(set(df_genome_search["virOTU"].to_list()))
print("number of virus asvs",len(vir_asv_list))

In [ ]:
df_genome2_from = df_genome.copy()
df_genome2_to = df_genome.copy()
df_genome2_from.rename(columns={"Families": "to_family", "virOTU": "from"}, inplace=True)
df_genome2_to.rename(columns={"Families": "from_family", "virOTU": "to"}, inplace=True)
df_genome2_from["left"]=1
df_genome2_to["right"]=1

In [ ]:
df_candidates_merge =pd.merge(df_ccm, df_genome2_from, how="outer", on=["from", "to_family"])
df_candidates_merge2 =pd.merge(df_candidates_merge, df_genome2_to, how="outer", on=["to", "from_family"])

In [ ]:
df_candidates_merge.columns

In [ ]:
df_candidates_merge2[(df_candidates_merge2["left"]==1)&(df_candidates_merge2["InCCMN"]==1)].shape

In [ ]:
df_candidates_merge2[(df_candidates_merge2["right"]==1)&(df_candidates_merge2["InCCMN"]==1)].shape

In [ ]:
table_prunded_network =df_candidates_merge2[((df_candidates_merge2["right"]==1)  | (df_candidates_merge2["left"]==1)) &(df_candidates_merge2["InCCMN"]==1)]

# Network with the egdes from the delivered tables

In [ ]:
table_prunded_network.shape

In [ ]:
table_prunded_network.columns

In [ ]:
table_prunded_network_save = table_prunded_network[['from', 'to', 'corr', 'from_clu', 'to_clu', 'p-value',
       'from_kingdom', 'to_kingdom', 'from_family', 'to_family']]

In [ ]:
table_prunded_network_save.to_csv("tables/table_prunded_network_without_pval.csv", index=False, sep=",")
table_prunded_network_save.shape

In [ ]:
table_prunded_network_save.describe()

# Network with significant prediction from the provided table

In [ ]:
table_prunded_network_save_significant = table_prunded_network_save[table_prunded_network_save["p-value"]<0.05]
table_prunded_network_save_significant.shape

In [ ]:
table_prunded_network_save_significant.to_csv("tables/table_prunded_network_significant.csv", index=False, sep=",")

In [ ]:
table_prunded_network_save_significant.describe()

In [ ]:
table_prunded_network_save_significant[table_prunded_network_save_significant["from_kingdom"]=="virus"].describe()

In [ ]:
table_prunded_network_save_significant[table_prunded_network_save_significant["to_kingdom"]=="virus"].describe()

In [ ]:
table_prunded_network_save_significant[table_prunded_network_save_significant["from_kingdom"]=="virus"]

In [ ]:
table_prunded_network_save_significant[table_prunded_network_save_significant["to_kingdom"]=="virus"]